In [1]:
import onnxruntime as ort
from onnxruntime_tools.quantization import quantize_static, QuantType
import cv2
import numpy as np
from pathlib import Path


In [2]:
input_name = "images"
class VideoDataLoader:
    
    def __init__(self, video_path):
        self.video_path = video_path
        self.cap = cv2.VideoCapture(video_path)
        self.input_size = (640, 640)
        
    
    def get_next(self):
        cap = self.cap
        ret = None
        if cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                return None
            inp = cv2.resize(frame, self.input_size)
            inp = inp.astype(np.float32) / 255.0
            inp = np.transpose(inp, (2, 0, 1))  # from HxWxC to CxHxW
            inp = np.expand_dims(inp, axis=0)
            ret = {input_name: inp}
        return ret
    
data_loader = VideoDataLoader("../input_videos/12057881_3840_2160_24fps.mp4")

In [3]:
model_path = Path("../yolo11s.onnx")
quant_model_path = str(model_path.parent / model_path.stem) + "_int8.onnx"

quantize_static(model_path, quant_model_path, data_loader, QuantType.QInt8)